# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

,Unnamed: 0,Country,Year,Debt,RGDP,GDP,dRGDP,GDPI,GDP1,GDP2,...,GDPalt,RGDP2alt,debtgdp,GDP3,GNI,lRGDP,lRGDP1,lRGDP2,dgcat,dgcat2
0,1,Australia,1946,5588.8,NaN,NaN,-3.557951,NaN,2935.0,NaN,...,NaN,NaN,190.419080,NaN,NaN,NaN,51108.960,NaN,Above 90%,Above 120%
1,2,Australia,1947,5534.2,NaN,NaN,2.459475,NaN,3121.0,NaN,...,NaN,NaN,177.321371,NaN,NaN,NaN,49290.528,NaN,Above 90%,Above 120%
2,3,Australia,1948,5580.4,NaN,NaN,6.437534,NaN,3747.0,NaN,...,NaN,NaN,148.929811,NaN,NaN,NaN,50502.816,NaN,Above 90%,Above 120%
3,4,Australia,1949,5656.0,NaN,NaN,6.611994,NaN,4324.0,4495.0,...,NaN,NaN,125.828699,NaN,NaN,NaN,53753.952,NaN,Above 90%,Above 120%
4,5,Australia,1950,5818.8,NaN,NaN,6.920201,NaN,5099.0,5299.0,...,NaN,NaN,109.809398,NaN,NaN,NaN,57308.160,NaN,Above 90%,90-120%


## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [10]:
import statsmodels.api as sm
import patsy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RR = pd.read_csv('https://gist.githubusercontent.com/vincentarelbundock/5409893/raw/a623f2f3bae027a0e51dd01ac5b70d44d909a7b9/RR-processed.csv')
RR.groupby('Country').size()
bins = ["0-30%","30-60%","60-90%","Above 90%"]
RR['dgcat'] = np.digitize(RR.debtgdp, [0,30,60,90,np.inf]) - 1
RR.dgcat = [bins[x] for x in RR.dgcat]

bins = ["0-30%","30-60%","60-90%","90-120%","Above 120%"]
RR['dgcat2'] = np.digitize(RR.debtgdp, [0,30,60,90,120,np.inf]) - 1
RR.dgcat2 = [bins[x] for x in RR.dgcat2]

RR.head(5)

y,X = patsy.dmatrices('dRGDP ~ dgcat', data=RR[['dRGDP', 'dgcat']].dropna())
print (sm.OLS(y,X).fit().summary())


#Code seems to not work in google colab or was written in python 2. 

                            OLS Regression Results                            
Dep. Variable:                  dRGDP   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     18.36
Date:                Tue, 08 Jan 2019   Prob (F-statistic):           1.22e-11
Time:                        00:23:07   Log-Likelihood:                -2927.9
No. Observations:                1175   AIC:                             5864.
Df Residuals:                    1171   BIC:                             5884.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              4.1735      0

### Assignment questions

After you've worked on some code, answer the following questions in this text block:


1. Today I became familiar with the logistics of being a lambda school student.

2.  Nothing has been challenging so far.

3.  Learning how to connect Colab with github.

4.  Definitely machine learning and neural nets.



## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about